In [ ]:
import perceval as pcvl
import perceval.lib.phys as phys

import numpy as np

In [2]:
pcvl.register_plugin("perceval_internal", silent=True)

True

In [3]:
from qiskit import QuantumCircuit

In [4]:
from perceval.converters import perceval_qiskit

In [5]:
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.barrier()
circuit.s(0)
circuit.ry(np.pi/4, 1)
circuit.cx(1, 2)
circuit.barrier()
circuit.swap(2, 1)
circuit.rz(np.pi/4, 2)
circuit.cx(0, 2)
circuit.draw()

┌───┐      ░    ┌───┐         ░                    
q_0: ┤ H ├──■───░────┤ S ├─────────░─────────────────■──
     └───┘┌─┴─┐ ░ ┌──┴───┴──┐      ░                 │  
q_1: ─────┤ X ├─░─┤ Ry(π/4) ├──■───░──X──────────────┼──
          └───┘ ░ └─────────┘┌─┴─┐ ░  │ ┌─────────┐┌─┴─┐
q_2: ───────────░────────────┤ X ├─░──X─┤ Rz(π/4) ├┤ X ├
                ░            └───┘ ░    └─────────┘└───┘

In [6]:
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [7]:
pc=perceval_qiskit.to_perceval(circuit, phys, heralded=True)
pcvl.pdisplay(pc, recursive=True, render_size=0.5)

0 [2, 3, 0, 1, 6, 7, 4, 5] [2, 3, 0, 1, 6, 7, 4, 5]


AttributeError: 'PredefinedCircuit' object has no attribute 'm'

In [ ]:
u=pc.compute_unitary(use_symbolic=False)

In [ ]:
ub = (pcvl.Circuit(2)
      // (0, symb.PS(phi=pcvl.Parameter("φ_a")))
      // symb.BS(theta=pcvl.Parameter("theta")))

pc_norm = pcvl.Circuit.decomposition(u, ub, shape="rectangle")
pcvl.pdisplay(pc_norm, compact=True, render_size=0.8)
